# Pipeline definition

## Preprocessing

- class to apply rolling means along real-valued columns of a given data frame

In [ ]:
class Preprocessing_RollingMean():
        
    def __init__(self, time_start=0, time_wind=3):
        if not(time_wind <= time_start):
            print('WARNING! The transform method can be applied,',
                  'however, the mean can not be taken over time_wind={} samples'.format(time_wind+1),
                  'for samples with indices smaller than time_wind={}.'.format(time_wind+1))
        self.time_start = time_start
        self.time_wind = time_wind
    
    def transform(self, df):
        
        """
        Compute the rolling mean along each column of df,
        starting at the self.time_start index row and
        using a time window of self.time_wind + 1.
        
        Input:
        df:   dataframe of shape samples x sensors
        
        Output:
        X:    dataframe of shape samples x sensors
        """
   
        X = []
        index = df.loc[time_start:,].index
        columns = df.columns
        for idx in index:
            x = df.loc[idx-self.time_wind:idx,:]
            x_mean = np.mean(np.array(x), axis=0)
            X.append(x_mean)
        
        X = pd.DataFrame(data=X, 
                         index=index,
                         columns=columns)
        
        return X

## Visualization

- functionality to visualize the used water distribution network, its sensor nodes and sensitive groups
- functionality to visualize arbitrary historical data (such as training, test and predicted data) at arbitrary sensors, based on a given time index
- functionality to visualize arbitrary historical data (such as training, test and predicted data) at arbitrary sensors, based on a given leak setting (depending on leak location and diameter)

In [ ]:
def plot_network_Hanoi(node_ids,
                       sensor_ids,
                       df_information,
                       wn,
                       name='Hanoi',
                       save_figs=False):
    
    """
    Input:
    node_ids:         list of node labels
    sensor_ids:       list of node labels of nodes which are sensors
    df_information:   data frame which holds information about the settings
    wn:               wntr water network
    name:             string of the name of the wntr water network
    save_figs:        boolean to determine whether creates grahps should be saved
    """
    
    # --- find out what node belongs to which sensitive group
    groups_per_node = dict()
    for node_id in node_ids:

        # find sensitive group of the node id
        filter_node_id = df_information['node ID'] == node_id
        index_node_id = list(df_information[filter_node_id].index)[0]
        group = df_information.loc[index_node_id, 'group']
        groups_per_node[node_id] = group
        
    # --- create color map  corresponding to each sensitive group
    # (required for network plot)
    nb_sensitive_features = len(set(groups_per_node.values()))
    node_color_map = cm.get_cmap(name='Blues', lut=1000) #coolwarms
    node_color_map = ListedColormap(node_color_map(np.linspace(0.3, 0.8, nb_sensitive_features)))

    # --- create node attributes corresponding to each sensitive group
    # (required for network plot)
    node_attributes = dict()
    node_attributes_sensors = dict()
    for node_id in node_ids:
        # this needs to be generalized to more or less sensitive groups
        if groups_per_node[node_id]=='group1':
            node_attributes[node_id] = 1/nb_sensitive_features
            if node_id in sensor_ids:
                node_attributes_sensors[node_id] = (0.6,0.6,0.6,0.5)
        if groups_per_node[node_id]=='group2':
            node_attributes[node_id] = 3/nb_sensitive_features
            if node_id in sensor_ids:
                node_attributes_sensors[node_id] = (0.6,0.6,0.6,0.5)
        if groups_per_node[node_id]=='group3':
            node_attributes[node_id] = 2/nb_sensitive_features
            if node_id in sensor_ids:
                node_attributes_sensors[node_id] = (0.6,0.6,0.6,0.5)

    
    # --- plot water network with sensor nodes and sensitve groups highlighted
    fig, ax = plt.subplots(1, 1,
                           # https://www.statology.org/subplot-size-matplotlib/
                           figsize=(10,5))
    ax.set_title('{} network and its sensor nodes and sensitive groups'.format(name),
                 fontsize='medium', pad=0)
    
    # this is a bit un-nice because of the plot_network function plots the network automatically
    wntr.graphics.plot_network(wn, 
                               node_attribute=node_attributes_sensors, 
                               node_cmap=node_color_map,
                               node_size=800,
                               node_labels=False,
                               link_width=0,
                               link_labels=False,
                               add_colorbar=False,
                               ax=ax)
    wntr.graphics.plot_network(wn, 
                               node_attribute=node_attributes, 
                               node_cmap=node_color_map,
                               node_size=350,
                               node_labels=True,
                               link_width=1,
                               link_labels=False,
                               add_colorbar=False,
                               ax=ax)
    
    if save_figs:
        fig.savefig('{}_sensornodes_{}_{}_{}.pdf'.format(name,
                                                         sensor_ids[0],
                                                         sensor_ids[1],
                                                         sensor_ids[2]),
                    format='pdf')
        
    return fig

In [ ]:
def plot_network_LTown(node_ids,
                       sensor_ids,
                       groups_per_node,
                       wn,
                       name='L-Town',
                       save_figs=False):
    
    """
    Input:
    node_ids:         list of node labels
    sensor_ids:       list of node labels of nodes which are sensors
    groups_per_node:  dictionary with key=node id, value=sensitive group
    wn:               wntr water network
    name:             string of the name of the wntr water network
    save_figs:        boolean to determine whether creates grahps should be saved
    """
        
    # --- create color map  corresponding to each sensitive group
    # (required for network plot)
    nb_sensitive_features = len(set(groups_per_node.values()))
    node_color_map = cm.get_cmap(name='Blues', lut=1000) #coolwarms
    node_color_map = ListedColormap(node_color_map(np.linspace(0.3, 0.8, nb_sensitive_features)))

    # --- create node attributes corresponding to each sensitive group
    # (required for network plot)
    node_attributes = dict()
    node_attributes_sensors = dict()
    for node_id in node_ids:
        # this needs to be generalized to more or less sensitive groups
        if groups_per_node[node_id]=='group1':
            node_attributes[node_id] = 1/nb_sensitive_features
            if node_id in sensor_ids:
                node_attributes_sensors[node_id] = (0.6,0.6,0.6,0.5)
        if groups_per_node[node_id]=='group2':
            node_attributes[node_id] = 3/nb_sensitive_features
            if node_id in sensor_ids:
                node_attributes_sensors[node_id] = (0.6,0.6,0.6,0.5)
        if groups_per_node[node_id]=='group3':
            node_attributes[node_id] = 2/nb_sensitive_features
            if node_id in sensor_ids:
                node_attributes_sensors[node_id] = (0.6,0.6,0.6,0.5)

    
    # --- plot water network with sensor nodes and sensitve groups highlighted
    fig, ax = plt.subplots(1, 1,
                           # https://www.statology.org/subplot-size-matplotlib/
                           figsize=(10,5))
    ax.set_title('{} network and its sensor nodes and sensitive groups'.format(name),
                 fontsize='medium', pad=0)
    
    # this is a bit un-nice because of the plot_network function plots the network automatically
    wntr.graphics.plot_network(wn, 
                               node_attribute=node_attributes_sensors, 
                               node_cmap=node_color_map,
                               node_size=75,
                               node_labels=False,
                               link_width=0,
                               link_labels=False,
                               add_colorbar=False,
                               ax=ax)
    wntr.graphics.plot_network(wn, 
                               node_attribute=node_attributes, 
                               node_cmap=node_color_map,
                               node_size=20,
                               node_labels=False,
                               link_width=1,
                               link_labels=False,
                               add_colorbar=False,
                               ax=ax)
    
    if save_figs:
        fig.savefig('{}_sensornodes_{}_{}_{}.pdf'.format(name,
                                                         sensor_ids[0],
                                                         sensor_ids[1],
                                                         sensor_ids[2]),
                    format='pdf')
        
    return fig

In [ ]:
def plot_data_per_timeindex(df,
                            sensor_ids,
                            start_ids,
                            end_ids,
                            thresholds=None,
                            show_legend=True):
    
    """
    Input:
    df:              data frame
                     which holds the to be plotted data for each sensor
    sensor_ids:      list of sensor ids of sensors 
                     of which the data should be visualized
    start_ids:       list of time ids of times
                     starting from which the data should be visualized
    end_ids:         list of time ids of times
                     up to which the data should be visualized
    thresholds:      dictionary with key=sensor id, value=threshold for decision gap
    show_legend:     boolean which indicates whether or not a legend should be shown
    """
    
    # --- plot data for each pair of start and end IDs
    for start_id, end_id in zip(start_ids, end_ids):
    
        # generate time index
        index_plot = pd.RangeIndex(start=start_id,
                                   stop=end_id)

        # -- plot data for each sensor
        plt.figure(figsize=(22,5))
        color_map = cm.get_cmap(name='coolwarm', lut=100)
        sub_color_map = color_map(np.linspace(1.0, 0.0, len(sensor_ids)))
        for index, (color, sensor_id) in enumerate(zip(sub_color_map,sensor_ids)):
            
            # define linestyle depending on 
            # availability of a threshold for a decision gap and
            # the number of grafs plotted already
            if not(thresholds == None) and (sensor_id in thresholds.keys()):
                    linestyle = ':'
            else:
                if index > 2:
                    linestyle = '--'
                else:
                    linestyle = '-'
            
            plt.plot(index_plot, df.loc[index_plot, sensor_id], 
                     label='sensor {}'.format(sensor_id), 
                     color=color, 
                     linestyle=linestyle)
                     #alpha=1-index/(2*len(sensor_ids)))

            # plot decision gap if required information is available ...
            if not(thresholds == None):
                # ... and the data required is the one we consider right now
                if sensor_id in thresholds.keys():
                    upper_decision_gap = df.loc[index_plot, sensor_id] + thresholds[sensor_id]
                    lower_decision_gap = df.loc[index_plot, sensor_id] - thresholds[sensor_id]
                    plt.fill_between(index_plot, upper_decision_gap, lower_decision_gap,
                                     label='decision gap', 
                                     color=color,
                                     alpha=0.2)

        title = 'Pressure at different sensors'
        plt.title(title)
        plt.xlabel('time (600s)')
        plt.ylabel('pressure')
        if show_legend:
            plt.legend()

    plt.show()    

In [ ]:
def plot_data_per_timeindex_and_sensor(dfs,
                                       sensor_ids,
                                       start_ids,
                                       end_ids,
                                       thresholds=None,
                                       threshold_key=None,
                                       show_legend=True):
    
    """
    Input:
    dfs:             dictionary with key=plot label, value=data frame
                     which holds the to be plotted data for each sensor
    sensor_ids:      list of sensor ids of sensors 
                     of which the data should be visualized
    start_ids:       list of time ids of times
                     starting from which the data should be visualized
    end_ids:         list of time ids of times
                     up to which the data should be visualized
    thresholds:      dictionary with key=sensor id, value=threshold for decision gap
    threshold_key:   string which is some of dfs.keys() 
                     which indicates around which plot to draw a decision gap
    show_legend:     boolean which indicates whether or not a legend should be shown
    """
    
    # --- plot data for each pair of start and end IDs
    for start_id, end_id in zip(start_ids, end_ids):
    
        # generate time index
        index_plot = pd.RangeIndex(start=start_id,
                                   stop=end_id)

        # -- plot data for each sensor
        plt.figure(figsize=(22,5))
        for index, sensor_id in enumerate(sensor_ids):

            plt.subplot(1, len(sensor_ids), index+1)

            # -- plot data for each data frame in dfs
            colors = ['grey', 'orange', 'blue']
            linestyles = ['-', ':', ':']
            alphas = [0.6, 1, 1]
            for key, color, linestyle, alpha in zip(dfs, colors, linestyles, alphas):

                plt.plot(index_plot, dfs[key].loc[index_plot, sensor_id], 
                         label=key, 
                         color=color, 
                         linestyle=linestyle, 
                         alpha=alpha)

                # plot decision gap if required information is available ...
                if not(thresholds == None) and not(threshold_key == None):
                    # ... and the data required is the one we consider right now
                    if (sensor_id in thresholds.keys()) and (key == threshold_key):
                        upper_decision_gap = dfs[key].loc[index_plot, sensor_id] + thresholds[sensor_id]
                        lower_decision_gap = dfs[key].loc[index_plot, sensor_id] - thresholds[sensor_id]
                        plt.fill_between(index_plot, upper_decision_gap, lower_decision_gap,
                                         label='decision gap', 
                                         color=color,
                                         alpha=0.2)

            title = 'Pressure at sensor {}'.format(sensor_id)
            plt.title(title)
            plt.xlabel('time (600s)')
            plt.ylabel('pressure at sensor {}'.format(sensor_id))
            if show_legend:
                plt.legend()

        plt.show()    

In [ ]:
def plot_data_per_setting(df,
                          df_information,
                          sensor_ids,
                          node_ids,
                          diameters,
                          setting_ids=None,
                          thresholds=None,
                          time_puffer=100,
                          show_legend=True,
                          zoom_leak=False,
                          print_report=False): 
    
    """
    Input:
    df:              data frame
                     which holds the to be plotted data for each sensor
    df_information:  data frame which holds information about the settings
    sensor_ids:      list of sensor ids of sensors 
                     of which the data should be visualized
    node_ids:        list of node ids of nodes
                     which belong to the settig that should be visualized
    diameters:       list of diameters
                     which belong to the settig that should be visualized
    setting_ids:     list of setting ids of settings 
                     (combinations of node_id and diameter)
                     which should be visualized
                          if==None, setting_ids is generated by
                          node_ids and diameters
    thresholds:      dictionary with key=sensor id, value=threshold for decision gap
    time_puffer:     float which indicates about how much the time index of the settings
                     should be extended before and after the settings for visualization
    show_legend:     boolean which indicates whether or not a legend should be shown
    zoom_leak:       boolean which indicates whether of not the data should be zoomed
                     to the leak in the specified settings
    print_report:    boolean which indicates whether or not intermediate results should be printed
    """
    
    # --- generate setting_ids (if necessary)
    if setting_ids == None:
        setting_ids = list()
        for node_id in node_ids:
            for diameter in diameters:
                condition_node = df_information['node ID'] == node_id
                condition_diameter = df_information['diameter'] == diameter
                setting_id = list(df_information[condition_node & condition_diameter].index)[0]
                setting_ids.append(setting_id)
        if print_report:
            print('Setting IDs where generated: {}'.format(setting_ids))
    
    # --- plot data for each setting
    for setting_id in setting_ids:
        
        # access setting information
        node_id = df_information.loc[setting_id,'node ID']
        diameter = df_information.loc[setting_id,'diameter']
        setting_start_id = df_information.loc[setting_id,'setting start ID']
        leak_start_id = df_information.loc[setting_id,'leak start ID']
        leak_end_id = df_information.loc[setting_id,'leak end ID']
        setting_end_id = df_information.loc[setting_id,'setting end ID']
        
        # generate time index
        if zoom_leak:
            index_plot = pd.RangeIndex(start=leak_start_id-time_puffer,
                                       stop=leak_end_id+time_puffer)
        else:
            index_plot = pd.RangeIndex(start=setting_start_id-time_puffer,
                                       stop=setting_end_id+time_puffer)
     
        # -- plot data for each sensor
        plt.figure(figsize=(22,5))
        color_map = cm.get_cmap(name='coolwarm', lut=100)
        sub_color_map = color_map(np.linspace(1.0, 0.0, len(sensor_ids)))
        for index, (color, sensor_id) in enumerate(zip(sub_color_map,sensor_ids)):
            
            # define linestyle depending on 
            # availability of a threshold for a decision gap and
            # the number of grafs plotted already
            if not(thresholds == None) and (sensor_id in thresholds.keys()):
                    linestyle = ':'
            else:
                if index > 2:
                    linestyle = '--'
                else:
                    linestyle = '-'
                
            plt.plot(index_plot, df.loc[index_plot, sensor_id], 
                     label='sensor {}'.format(sensor_id), 
                     color=color, 
                     linestyle=linestyle)
                     #alpha=1-index/(2*len(sensor_ids)))

            # plot decision gap if required information is available ...
            if not(thresholds == None): 
                # ... and the data required is the one we consider right now
                if sensor_id in thresholds.keys():
                    upper_decision_gap = df.loc[index_plot, sensor_id] + thresholds[sensor_id]
                    lower_decision_gap = df.loc[index_plot, sensor_id] - thresholds[sensor_id]
                    plt.fill_between(index_plot, upper_decision_gap, lower_decision_gap,
                                     label='decision gap', 
                                     color=color,
                                     alpha=0.2)

            # plot start and end of leak
            plt.scatter([leak_start_id, leak_end_id], 
                        [df.loc[leak_start_id, sensor_id],
                         df.loc[leak_end_id, sensor_id]],
                         label='leak start and end',
                         color=color,
                         marker='o')

        title = 'Pressure at different sensors'
        title += '\nLeak location: Node {}'.format(node_id)
        title += '\nLeak diameter: {} cm'.format(diameter)
        plt.title(title)
        plt.xlabel('time (600s)')
        plt.ylabel('pressure')
        if show_legend:
            plt.legend()

    plt.show()    

In [ ]:
def plot_data_per_setting_and_sensor(dfs,
                                     df_information,
                                     sensor_ids,
                                     node_ids,
                                     diameters,
                                     setting_ids=None,
                                     thresholds=None,
                                     threshold_key=None,
                                     leak_key=None,
                                     time_puffer=100,
                                     show_legend=True,
                                     zoom_leak=False,
                                     print_report=False): 
    
    """
    Input:
    dfs:             dictionary with key=plot label, value=data frame
                     which holds the to be plotted data for each sensor
    df_information:  data frame which holds information about the settings
    sensor_ids:      list of sensor ids of sensors 
                     of which the data should be visualized
    node_ids:        list of node ids of nodes
                     which belong to the settig that should be visualized
    diameters:       list of diameters
                     which belong to the settig that should be visualized
    setting_ids:     list of setting ids of settings 
                     (combinations of node_id and diameter)
                     which should be visualized
                          if==None, setting_ids is generated by
                          node_ids and diameters
    thresholds:      dictionary with key=sensor id, value=threshold for decision gap
    threshold_key:   string which is some of dfs.keys() 
                     which indicates around which plot to draw a decision gap
    leak_key:        string which is some of dfs.keys() 
                     which indicates on which plot to draw the leak start and end
    time_puffer:     float which indicates about how much the time index of the settings
                     should be extended before and after the settings for visualization
    show_legend:     boolean which indicates whether or not a legend should be shown
    zoom_leak:       boolean which indicates whether of not the data should be zoomed
                     to the leak in the specified settings
    print_report:    boolean which indicates whether or not intermediate results should be printed
    """
    
    # --- generate setting_ids (if necessary)
    if setting_ids == None:
        setting_ids = list()
        for node_id in node_ids:
            for diameter in diameters:
                condition_node = df_information['node ID'] == node_id
                condition_diameter = df_information['diameter'] == diameter
                setting_id = list(df_information[condition_node & condition_diameter].index)[0]
                setting_ids.append(setting_id)
        if print_report:
            print('Setting IDs where generated: {}'.format(setting_ids))
    
    # --- plot data for each setting
    for setting_id in setting_ids:
        
        # access setting information
        node_id = df_information.loc[setting_id,'node ID']
        diameter = df_information.loc[setting_id,'diameter']
        setting_start_id = df_information.loc[setting_id,'setting start ID']
        leak_start_id = df_information.loc[setting_id,'leak start ID']
        leak_end_id = df_information.loc[setting_id,'leak end ID']
        setting_end_id = df_information.loc[setting_id,'setting end ID']
        
        # generate time index
        if zoom_leak:
            index_plot = pd.RangeIndex(start=leak_start_id-time_puffer,
                                       stop=leak_end_id+time_puffer)
        else:
            index_plot = pd.RangeIndex(start=setting_start_id-time_puffer,
                                       stop=setting_end_id+time_puffer)
     
        # -- plot data for each sensor
        plt.figure(figsize=(22,5))
        for index, sensor_id in enumerate(sensor_ids):
            
            plt.subplot(1, len(sensor_ids), index+1)
            
            # -- plot data for each data frame in dfs
            colors = ['grey', 'orange', 'blue']
            linestyles = ['-', ':', ':']
            alphas = [0.6, 1, 1]
            for key, color, linestyle, alpha in zip(dfs, colors, linestyles, alphas):
                
                plt.plot(index_plot, dfs[key].loc[index_plot, sensor_id], 
                         label=key,
                         color=color, 
                         linestyle=linestyle, 
                         alpha=alpha)
            
                # plot decision gap if required information is available ...
                if not(thresholds == None) and not(threshold_key == None):
                    # ... and the data required is the one we consider right now
                    if (sensor_id in thresholds.keys()) and (key == threshold_key):
                        upper_decision_gap = dfs[key].loc[index_plot, sensor_id] + thresholds[sensor_id]
                        lower_decision_gap = dfs[key].loc[index_plot, sensor_id] - thresholds[sensor_id]
                        plt.fill_between(index_plot, upper_decision_gap, lower_decision_gap,
                                         label='decision gap', 
                                         color=color,
                                         alpha=0.2)
                        
                # plot start and end of leak if required information is available ...
                if not(leak_key == None):
                    # ... and the data required is the one we consider right now
                    if key == leak_key:
                        plt.scatter([leak_start_id, leak_end_id], 
                                    [dfs[key].loc[leak_start_id, sensor_id],
                                     dfs[key].loc[leak_end_id, sensor_id]],
                                     label='leak start and end',
                                     color=color,
                                     marker='o')

            title = 'Pressure at sensor {}'.format(sensor_id)
            title += '\nLeak location: Node {}'.format(node_id)
            title += '\nLeak diameter: {} cm'.format(diameter)
            plt.title(title)
            plt.xlabel('time (600s)')
            plt.ylabel('pressure at sensor {}'.format(sensor_id))
            if show_legend:
                plt.legend()
            
        plt.show()    

## Regression - Virtual Sensors

- class to train and test a multi regression model, i.e., a regression model per column of a given data frame

In [ ]:
class MultiRegression():
    
    def __init__(self, regressor):
        
        """
        Input:
        regressor:  regression class
        """
        
        self.regressor = regressor
        self.regressors = dict()
        
    def fit(self, X_train, Y_train, print_coeff=False):
        
        """
        Train regression model (virtual sensor) per sensor node
        based on the (pressure) inputs of the other sensor nodes.
        
        Input:
        X_train:      dataframe of training (pressure) inputs 
                      of shape samples x sensors
        Y_train:      dataframe of training (pressure) labels 
                      of shape samples x sensors
        print_coeff:  boolean which indicates whether trained coefficients
                      should be printed or not
        """
        
        # --- train one regression model (virtual sensor) per sensor node
        sensor_ids = list(X_train.columns)
        for node in sensor_ids:
            
            # -- extract sensor node specific training data
            columns = sensor_ids.copy()
            columns.remove(node)
            X_train_node = X_train.loc[:,columns] 
            y_train_node = Y_train.loc[:,[node]]
            
            # -- train regression model (virtual sensor)
            model = self.regressor()
            model.fit(X_train_node, y_train_node)
            if print_coeff:
                print('Model for sensor node {}:\n'\
                      'Coef.: {}, Intercept: {}.'.format(node,
                                                         model.coef_,
                                                         model.intercept_))
                
            # -- store trained regression model (virtual sensor) 
            self.regressors[node] = dict()
            self.regressors[node]['regressor'] = model
            self.regressors[node]['coef_'] = model.coef_
            self.regressors[node]['intercept_'] = model.intercept_
        
    def predict(self, X_test):
        
        """
        Predict (pressure) output per sensor node
        based on the (pressure) inputs of the other sensor nodes.
        
        Input:
        X_test:      dataframe of test (pressure) inputs 
                     of shape samples x sensors
        
        Output:
        Y_pred:      dataframe of test (pressure) labels 
                     of shape samples x sensors
        """
        
        # --- predict output (of virtual sensor) per sensor node
        sensor_ids = list(X_test.columns)
        Y_pred = pd.DataFrame(index=X_test.index,
                              columns=X_test.columns)
        for node in sensor_ids:
            
            # -- extract sensor node specific test data
            columns = sensor_ids.copy()
            columns.remove(node)
            X_test_node = X_test.loc[:,columns] 
            
            # -- access trained regression model (virtual sensor)
            model = self.regressors[node]['regressor']
            
            # -- predict output (of virtual sensor)
            y_pred = model.predict(X_test_node)
            y_pred = pd.DataFrame(data=y_pred,
                                  index=X_test_node.index,
                                  columns=[node])
            
            # -- store regression prediction (of virtual sensors)
            Y_pred.loc[:,node] = y_pred
            
        return Y_pred
    
    def score(self, X_test, Y_test, print_all_scores=False):
        
        """
        Compute R2 score and RMSE per sensor node.
        
        Input:
        X_test:             dataframe of test (pressure) inputs 
                            of shape samples x sensors
        Y_test:             dataframe of test (pressure) labels 
                            of shape samples x sensors
        print_all_scores:   boolean which indicates whether scores
                            should be printed or not
                            
        Output:
        mean_r2:            mean R2 score over all sensor nodes
                            and over all samples
        mean_rmse:          mean RMSE over all sensor nodes
                            over all samples
        """
        
        # --- predict output (of virtual sensor) per sensor node
        Y_pred = self.predict(X_test)
        
        # --- calculate scores per sensor node
        sensor_ids = list(X_test.columns)
        scores = pd.DataFrame(index=Y_test.columns,
                              columns=['rmse','r2'])
        for node in sensor_ids:
            
            # -- extract sensor node specific test and predicted data
            y_test = Y_test.loc[:,node]
            y_pred = Y_pred.loc[:,node]
            
            # -- store scores
            scores.loc[node,'rmse'] = mean_squared_error(y_test, y_pred, 
                                                         squared=False)
            scores.loc[node,'r2'] = r2_score(y_test, y_pred)
        
        if print_all_scores:
            print('All scores:\n', scores)
        mean_r2 = scores.loc[:,'r2'].mean()
        mean_rmse = scores.loc[:,'rmse'].mean()
        
        return mean_r2, mean_rmse

## Classification - Leak Dectector(s)

- class to apply a threshold based classfication model that can be integrated into the multi classification resp. ensemble classification class
- class to train and test a multi classfication model, i.e., a classification model per column of a given data frame, resp. an ensemble classification model
- different subclasses of the multi classification resp. ensemble classification class according to different training algorithms

In [ ]:
# some minor functions
def log_on_R(x):
    if x <= 0:
        return -1 * np.inf
    else:
        return np.log(x)

def sigmoid(x,b):
    return 1/(1+np.exp(-b*x))

def dx_sigmoid(x,b):
    return b * sigmoid(x,b) * (1-sigmoid(x,b))

def rates(y,f):
    #Input:
    #y, f:   dataframes of shape nodes x 1
    #        and with the same column name
    if float(y.sum()) != 0:
        rate = (y*f).sum() / y.sum()
        return float(rate)
    else:
        return '-'

def TPR(y_test,y_pred):
    #Input:
    #y_test, y_pred:   dataframes of shape nodes x 1
    #                  and with the same column name
    return rates(y_test,y_pred)

def FPR(y_test,y_pred):
    return rates(1-y_test,y_pred)

def FNR(y_test,y_pred):
    return rates(y_test,1-y_pred)

def TNR(y_test,y_pred):
    return rates(1-y_test,1-y_pred)

def PPV(y_test,y_pred):
    return rates(y_pred,y_test)

def ACC(y_test,y_pred):
    acc = (y_test*y_pred).sum() + ((1-y_test)*(1-y_pred)).sum()
    acc /= len(y_test)
    return float(acc)

def dx_ACC(y_test,dx_y_pred):
    dx_acc = (((2*y_test)-1)*dx_y_pred).sum()
    dx_acc /= len(y_test)
    return float(dx_acc)

def Cov(x_sen,y_pred):
    #Input:
    #x_sen, y_pred:   series of the shape nodes x 
    return x_sen.cov(y_pred)

In [ ]:
# threshold classification classes 
# (based on residual inputs)

class ThresholdClassification():
    
    def __init__(self, threshold):
        self.threshold = threshold
        
    # no fit method since this model is used in an esemble 
    # and training techniques might optimize over all esemble learner
        
    def predict(self, x_test):
        
        """
        Predict (leak y/n) output for one sensor node.
        
        Input:
        x_test:   series of test (pressure) residual inputs
        
        Output:
        y_pred:   series of test (leak y/n) outputs
        """
        
        find_leak = lambda x : 1 if (x > self.threshold) else 0
        y_pred = x_test.apply(find_leak)
        return y_pred
    
class ThresholdClassificationApproximation():
    
    def __init__(self,threshold):
        self.threshold = threshold
        
    # no fit method since this model is used in an esemble 
    # and training techniques might optimize over all esemble learner
        
    def predict(self, x_test, b_sigmoid):
        
        """
        Predict approximated (leak y/n) output for one sensor node.
        
        Input:
        x_test:   series of test (pressure) residuals inputs
        
        Output:
        y_pred:   series of test (leak y/n) outputs
        """
        
        y_pred = x_test.apply(lambda x: x - self.threshold)
        y_pred = y_pred.apply(sigmoid, b=b_sigmoid)
        return y_pred

In [ ]:
# threshold classification ensemble super classes
# (based on residual inputs)

class EnsembleThresholdClassification():
    
    def __init__(self, classifier, classifier_approx):
        
        """
        Input:
        classifier:          residual based threshold classifier class
        classifier_approx:   residual based approximative threshold 
                             classifier class
        """
        
        self.classifier = classifier
        self.classifier_approx = classifier_approx
        self.thresholds = dict()
        self.classifiers = dict()
        self.classifiers_approx = dict()
        
    def fit(self):
        print('No fitting method implemented yet')
        
    def fit_model(self, thresholds, print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector)
        per sensor node
        based on the trained thresholds in .fit() method.
        
        Input:
        thresholds:    dictionary with keys:node, values:(trained) threshold
        print_coeff:   boolean which indicates whether trained coefficients
                       should be printed or not
        """
        
        # -- store trained classification model (leak detector) per sensor node
        if print_coeff:
            print('\nFinal thresholds:')
        for node, threshold in thresholds.items():
            if print_coeff:
                print('Sensor: {}, Threshold: {}.'.format(node,threshold))
            model = self.classifier(threshold=threshold)
            model_approx = self.classifier_approx(threshold=threshold)
            self.classifiers[node] = dict()
            self.classifiers[node]['classifier'] = model
            self.classifiers[node]['threshold'] = threshold
            self.classifiers_approx[node] = dict()
            self.classifiers_approx[node]['classifier'] = model_approx
            self.classifiers_approx[node]['threshold'] = threshold
        
    def predict(self, 
                X_test,
                keys_list=None,
                thresholds_array=None):
        
        """
        Predict (leak y/n) output per sensor node 
        and as an ensemble (leak y/n) output over all sensors
        based on the (pressure) residual inputs per sensor node.
        
        Input:
        X_test:             dataframe of test (pressure) residual inputs 
                            of shape samples x sensors
        keys_list:          None or list of keys corresponding to each sensor node
                            of shape sensors x 1
                            and in the order of the corresponding thresholds 
                            in the thresholds_array
                            (if None, self.thresholds.keys() is used)
        thresholds_array:   None or numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        
        Output:
        Y_pred:             dataframe of test (leak y/n) labels 
                            of shape samples x sensors
        y_pred:             dataframe of test (leak y/n) labels
                            of shape samples x 1
                            
        For training purposes, this method is used for specified 
        keys_list and thresholds_array,
        for prediction purposes, these remain None and
        self.thresholds 
        is automatically used instead.
        """
        
        if (keys_list!=None) and (list(thresholds_array)!=None):
            thresholds = dict(zip(keys_list, thresholds_array))
        else:
            thresholds = self.thresholds
        
        # --- predict output (of leak detector) per sensor node
        sensor_ids = list(X_test.columns)
        Y_pred = pd.DataFrame(index=X_test.index,
                              columns=X_test.columns)
        for node in sensor_ids:
            
            # -- extract sensor node specific test data
            x_test_node = X_test.loc[:,node] 
            
            # -- access (trained) classification model (leak detector)
            threshold = thresholds[node]
            model = self.classifier(threshold=threshold)
            
            # -- predict output (of leak detector)
            y_pred = model.predict(x_test_node)
            
            # -- store classification prediction (of leak detector)
            Y_pred.loc[:,node] = y_pred
            
        # --- predict output (of leak detector) over all sensor nodes
        # predict a leak if at last one classification model 
        # per node predicts a leak
        Y_pred['sum'] =  Y_pred.agg(func='sum', axis=1)
        find_leak = lambda su : 1 if (su >= 1) else 0
        y_pred = Y_pred['sum'].apply(find_leak)
        y_pred = pd.DataFrame(data=y_pred,
                              index=y_pred.index).rename({'sum':'y'},
                                                         axis='columns')
                
        return Y_pred, y_pred
    
    def predict_approx(self, 
                       X_test, 
                       keys_list=None,
                       thresholds_array=None,
                       b_sigmoid=100, 
                       sum_threshold=0.8):
        
        """
        Predict approximated (leak y/n) output per sensor node 
        and as an ensemble (leak y/n) output over all sensors
        based on the (pressure) residual inputs per sensor node.
        
        Input:
        X_test:             dataframe of test (pressure) residual inputs 
                            of shape samples x sensors
        keys_list:          None or list of keys corresponding to each sensor node
                            of shape sensors x 1
                            and in the order of the corresponding thresholds 
                            in the thresholds_array
                            (if None, self.thresholds.keys() is used)
        thresholds_array:   None or numpy array of thresholds used for each sensor node
                            of shape nodes x 1
                            (if None, self.thresholds.keys() is used)
        b_sigmoid:          positive float that indicates 
                            the b in the sigmoid function
        sum_threshold:      positive float that indicates
                            when the sum of outputs over all sensor nodes indicates a leak
        
        Output:
        Y_pred:             dataframe of test (leak y/n) labels 
                            of shape samples x sensors
        y_pred:             dataframe of test (leak y/n) labels
                            of shape samples x 1
                            
        For training purposes, this method is used for specified 
        keys_list and thresholds_array,
        for prediction purposes, these remain None and
        self.thresholds 
        is automatically used instead.
        """
        
        if (keys_list!=None) and (list(thresholds_array)!=None):
            thresholds = dict(zip(keys_list, thresholds_array))
        else:
            thresholds = self.thresholds
        
        # --- approximately predict output (of leak detector) per sensor node
        sensor_ids = list(X_test.columns)
        Y_pred = pd.DataFrame(index=X_test.index,
                              columns=X_test.columns)
        for node in sensor_ids:
            
            # -- extract sensor node specific test data
            x_test_node = X_test.loc[:,node] 
            
            # -- access (trained) approximative classification model (leak detector)
            threshold = thresholds[node]
            model = self.classifier_approx(threshold=threshold)
            
            # -- predict output (of leak detector)
            y_pred = model.predict(x_test_node, b_sigmoid)
            
            # -- store classification prediction (of leak detector)
            Y_pred.loc[:,node] = y_pred
            
        # --- approximately predict output (of leak detector) over all sensor nodes
        # predict a leak if all classification models over all nodes
        # in sum predicts a value larger than sum_thresholds
        Y_pred['sum'] = Y_pred.agg(func='sum', axis=1)
        y_pred = Y_pred['sum'].apply(lambda su: su - sum_threshold)
        y_pred = y_pred.apply(sigmoid, b=b_sigmoid)
        y_pred = pd.DataFrame(data=y_pred,
                              index=y_pred.index).rename({'sum':'y'},
                                                         axis='columns')
                
        return Y_pred, y_pred
    
    def visualize_predict_approx(self,
                                 X_test,
                                 y_test,
                                 keys_list=None,
                                 thresholds_array=None,
                                 b_sigmoid=100,
                                 sum_threshold=0.8,
                                 iloc_index_start=0,
                                 iloc_index_end=1000):
        
        """
        Visualize sum of non-approximated and approximated (leak y/n) outputs over all sensors
        based on the (pressure) residual inputs per sensor node.
        
        Input:
        X_test:             dataframe of test (pressure) residual inputs 
                            of shape samples x sensors
        y_test:             dataframe of test (leak y/n) labels 
                            of shape samples x 1
        keys_list:          List of keys corresponding to each sensor node
                            of shape sensors x 1
                            and in the order of the corresponding thresholds 
                            in the thresholds_array
        thresholds_array:   Numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        b_sigmoid:          positive float that indicates 
                            the b in the sigmoid function
        sum_threshold:      positive float that indicates
                            when the sum of outputs over all sensor nodes indicates a leak
        iloc_index_start    index between 0 and len(X_test)
        iloc_index_end      index between iloc_index_start and len(X_test)
        """

        # --- (approximately) predict output (of leak detector) 
        # --- per sensor node and over all sensors
        Y_pred, y_pred = self.predict(X_test,
                                      keys_list=keys_list,
                                      thresholds_array=thresholds_array)
        Y_pred_a, y_pred_a = self.predict_approx(X_test,
                                                 keys_list=keys_list,
                                                 thresholds_array=thresholds_array,
                                                 b_sigmoid=b_sigmoid, 
                                                 sum_threshold=sum_threshold)

        # --- plot true labels, 
        # --- sum of (approximated) predicted outputs over all sensor nodes and
        # --- sum threshold
        nb_sensors = len(Y_pred.columns)-1
        
        # access true labels, (approximated) predictions and sum threshold
        Y = Y_pred.sort_index()
        Y = Y.iloc[iloc_index_start:iloc_index_end,:]
        Y_a = Y_pred_a.sort_index()
        Y_a = Y_a.iloc[iloc_index_start:iloc_index_end,:]
        y = y_clas_train.sort_index()
        y = y.iloc[iloc_index_start:iloc_index_end,:]
        t = [1 for y in range(len(y))]
        t_a = [sum_threshold for y in range(len(y))]
        
        # true labels vs. sum of predicted outputs
        plt.figure(figsize=(20,10))
        plt.scatter(list(range(len(Y))), Y['sum'])
        plt.scatter(list(range(len(y))), y*nb_sensors, marker='.')
        plt.plot(list(range(len(y))), t, color='red', linestyle='--')
        plt.title('True labels vs. sum of predicted outputs')
        plt.show()

        # true labels vs. sum of (approximated) predicted outputs
        plt.figure(figsize=(20,10))
        plt.scatter(list(range(len(Y_a))), Y_a['sum'])
        plt.scatter(list(range(len(y))), y*nb_sensors, marker='.')
        plt.plot(list(range(len(y))), t_a, color='red', linestyle='--')
        plt.title('True labels vs. sum of approximated predicted outputs')
        plt.show()
        
        # --- evaluate (approximated) predictions
        tpr = TPR(y_test,y_pred)
        fpr = FPR(y_test,y_pred)
        acc = ACC(y_test,y_pred)

        print('Exact TPR:', tpr)
        print('Exact FPR:', fpr)
        print('Exact ACC:', acc)
        
        tpr = TPR(y_test,y_pred_a)
        fpr = FPR(y_test,y_pred_a)
        acc = ACC(y_test,y_pred_a)

        print('Approximated TPR:', tpr)
        print('Approximated FPR:', fpr)
        print('Approximated ACC:', acc)
    
    def dx_predict_approx(self, 
                          X_test, 
                          keys_list=None,
                          thresholds_array=None,
                          b_sigmoid=100, 
                          sum_threshold=0.8):
        
        """
        Compute gradient with respect to thresholds per sensor node
        of approximated (leak y/n) output prediction per sensor node 
        based on the (pressure) residual inputs per sendor node.
        
        Input:
        X_test:             dataframe of test (pressure) residual inputs 
                            of shape samples x sensors
        keys_list:          None or list of keys corresponding to each sensor node
                            of shape sensors x 1
                            and in the order of the corresponding thresholds 
                            in the thresholds_array
                            (if None, self.thresholds.keys() is used)
        thresholds_array:   None or numpy array of thresholds used for each sensor node
                            of shape nodes x 1
                            (if None, self.thresholds.keys() is used)
        b_sigmoid:          positive float that indicates 
                            the b in the sigmoid function
        sum_threshold:      positive float that indicates
                            when the sum of sensor node outputs indicates a leak
        
        Output:
        dx_Y_pred:          dataframe of partial derivatives of the
                            test (leak y/n) labels 
                            with respect to the thresholds per sensor node
                            of shape samples x sensors
                           
        For training purposes, this method is used for specified 
        keys_list and thresholds_array,
        for prediction purposes, these remain None and
        self.thresholds 
        is automatically used instead.
        """
        
        # --- approximately predict output (of leak detector) 
        # --- per sensor node and over all sensors
        Y_pred, y_pred = self.predict_approx(X_test,
                                             keys_list=keys_list,
                                             thresholds_array=thresholds_array,
                                             b_sigmoid=b_sigmoid, 
                                             sum_threshold=sum_threshold)
        
        # --- compute partial derivatives of approximately predicted output 
        # --- wrt. each sensor node's threshold
        # -- precompute factors which are the same for each partial derivative
        # (this is the "outer" derivative)
        factor = - b_sigmoid**2 * y_pred * (1 - y_pred)
        
        # -- precompute factors which are sensor specific
        # (this is the "inner" derivative)
        factor_nodes = Y_pred * (1 - Y_pred)
        
        # -- combine outer and inner derivative per sensor node
        sensor_ids = list(X_test.columns)
        dx_Y_pred = pd.DataFrame(index=X_test.index,
                                 columns=X_test.columns)
        for node in sensor_ids:
            
            # rename first factor to make pandas series multiplication possible
            factor_node_1 = factor.rename({'y':node},
                                          axis='columns')
            # access inner derivative of approximately predicted output 
            # wrt. to sensors's threshold
            factor_node_2 = factor_nodes.loc[:,[node]]
            
            # store partial derivatives of approximately predicted output 
            dx_Y_pred.loc[:,node] = factor_node_1 * factor_node_2
            
        return dx_Y_pred
            
    def score(self, 
              X_test, 
              y_test,
              keys_list=None,
              thresholds_array=None,
              print_all_scores=False):
        
        """
        Compute TPR, FNR, FPR, TNR, PPV and relative amout of positive predictions
        with respect to ensemble (leak y/n) output over all sensors
        based on the (pressure) residual inputs per sensor node.
        
        Input:
        X_test:             dataframe of test (pressure) residual inputs 
                            of shape samples x sensors
        y_test:             dataframe of test (leak y/n) labels 
                            of shape samples x 1
        keys_list:          None or list of keys corresponding to each sensor node
                            of shape sensors x 1
                            and in the order of the corresponding thresholds 
                            in the thresholds_array
                            (if None, self.thresholds.keys() is used)
        thresholds_array:   None or numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        print_all_scores:   boolean which indicates whether scores
                            should be printed or not
                            
        Output:
        tpr:                float of TPR over all samples
        fnr:                float of FNR over all samples
        fpr:                float of FPR over all samples
        tnr:                float of TNR over all samples
        ppv:                float of PPV score over all samples
        acc:                float of accuracy over all samples
        pos_pred:           float of amount of positive predictions 
                            over all samples
        """
        
        # --- predict output (of leak detector) over all sensor nodes
        _, y_pred = self.predict(X_test,
                                 keys_list=keys_list,
                                 thresholds_array=thresholds_array)
        
        # --- calculate scores
        tpr = TPR(y_test,y_pred) # tp/(tp+fn)
        fnr = FNR(y_test,y_pred) # fn/(tp+fn)
        fpr = FPR(y_test,y_pred) # fp/(fp+tn)
        tnr = TNR(y_test,y_pred) # tn/(fp+tn)
        ppv = PPV(y_test,y_pred) # tp/(tp+fp)
        acc = ACC(y_test,y_pred) # (tp+tn)/(tp+fn+fp+tn)
        pos_pred = float(y_pred.sum())/len(y_pred) # (tp+fp)/(tp+fn+fp+tn)
        
        if print_all_scores:
            print('TPR:', tpr)
            print('FNR = 1 - TPR:', fnr)
            print('FPR:', fpr)
            print('TNR = 1 - FPR:', tnr)
            print('PPV:', ppv)
            print('Acc.:', acc)
            print('Rel. pos. predictions:', pos_pred)
            print('TPR - FPR:', tpr-fpr)
        
        return tpr, fnr, fpr, tnr, ppv, acc, pos_pred
    
    def score_fairness(self, 
                       X_test, 
                       X_sen_test, 
                       y_test, 
                       keys_list=None,
                       thresholds_array=None,
                       print_all_scores=False):
        
        """
        Compute TPR, equal opportunity and disparate impact
        with respect to ensemble (leak y/n) output over all sensors
        based on the (pressure) residual inputs per sensor node.
        
        Input:
        X_test:             dataframe of test (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_test:         dataframe of test sensitive (feature) inputs 
                            of shape samples x sensitive features
        y_test:             dataframe of test (leak y/n) labels 
                            of shape samples x 1
        keys_list:          None or list of keys corresponding to each sensor node
                            of shape sensors x 1
                            and in the order of the corresponding thresholds 
                            in the thresholds_array
                            (if None, self.thresholds.keys() is used)
        thresholds_array:   None or numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        print_all_scores:   boolean which indicates whether scores
                            should be printed or not
                            
        Output:
        TPRs_list:          dict with key:sensitive group, 
                            value:TPR over all samples 
        eo:                 float of equal opportunity score
        di:                 float of disparate impact score
        """
        
        # --- compute TPR per sensitive group
        sensitive_features = list(X_sen_test.columns)
        TPRs_dict = dict()
        for sensitive_feature in sensitive_features:

            filter_group = X_sen_test[sensitive_feature] == 1
            tpr,_,_,_,_,_,_ = self.score(X_test[filter_group], 
                                         y_test[filter_group],
                                         keys_list=keys_list,
                                         thresholds_array=thresholds_array)
            TPRs_dict[sensitive_feature] = tpr

        # --- compute fairness scores
        TPRs_list = list(TPRs_dict.values())
        eo = max(TPRs_list) - min(TPRs_list)
        di = min(TPRs_list)/max(TPRs_list)

        if print_all_scores:
            print('TPRs:\n', TPRs_dict)
            print('Equal opportunity score (< epsilon), e.g. < 0.2:\n', eo)
            print('Disparate impact score (> 1 - epsilon), e.g. > 0.8:\n', di)
            # note that (1 - di) * max(TPRs_list) == eo

        return TPRs_dict, eo, di

In [ ]:
# threshold classification ensemble sub classes
# (based on residual inputs)
class ETC_hyperparameter(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='H',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
        
    def fit(self, 
            X_train, 
            factor=1, 
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector)
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by choosing some procentual amount of the largest training error 
        per sensor node.
        
        Input:
        X_train:      dataframe of training (pressure) residual inputs 
                      of shape samples x sensors
        factor:       positive float that is the procentual amount 
                      of the largest training error that should be used
        print_coeff:  boolean which indicates whether trained coefficients
                      should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by finding the largest training error per node
        sensor_ids = list(X_train.columns)
        for node in sensor_ids:

            # -- train classification model (leak detector)
            X_train_node = X_train.loc[:,node]
            threshold = factor * X_train_node.max()
            self.thresholds[node] = threshold
            
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)
            
class ETC_optimizeFTPR_db(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='T-F-PR',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
            
    def fit(self, 
            X_train,
            X_sen_train,
            y_train,
            start_thresholds,
            b_sigmoid=100, 
            sum_threshold=0.8,
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the db. approximation of the objective F = -TPR + FPR 
        over these thresholds.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of training sensitive (feature) inputs 
                            of shape samples x sensitive features
                            (only required for evaluation if print_coeff=True)
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        b_sigmoid:          positive float that indicates 
                            the b in the sigmoid function
        sum_thresholds:     positive float that indicates
                            when the sum of sensor node outputs indicates a leak
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective and objective's gradient
        def F(thresholds_array):
            _, y_pred = self.predict_approx(X_train, 
                                            keys_list=list(start_thresholds.keys()),
                                            thresholds_array=thresholds_array, 
                                            b_sigmoid=b_sigmoid,
                                            sum_threshold=sum_threshold)
            F = -TPR(y_train,y_pred) + FPR(y_train,y_pred)
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective information:')
                print('TPR:', TPR(y_train,y_pred))
                print('FPR:', FPR(y_train,y_pred))
                print('F:  ', F)
                
                print('\nCurrent fairness information:')
                sensitive_features = list(X_sen_train.columns)
                for sensitive_feature in sensitive_features:
                    x_sen_train = X_sen_train.loc[:, sensitive_feature]
                    cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                    print('Group:', sensitive_feature)
                    print('  cov:', cov)
        
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
                
            return np.array(F)
            
        def gradF(thresholds_array):
            dx_Y_pred = self.dx_predict_approx(X_train,
                                               keys_list=list(start_thresholds.keys()),
                                               thresholds_array=thresholds_array, 
                                               b_sigmoid=b_sigmoid,
                                               sum_threshold=sum_threshold)
            gradF = list()
            sensor_ids = list(X_train.columns)
            for node in sensor_ids:
                dx_y_pred = dx_Y_pred.loc[:,node]
                dx_y_pred = pd.DataFrame(data=dx_y_pred,
                                         index=dx_y_pred.index)
                dx_y_pred = dx_y_pred.rename({node:'y'},
                                              axis='columns')
                partialF = -TPR(y_train,dx_y_pred) + FPR(y_train,dx_y_pred)
                gradF.append(partialF)
                
            if print_coeff:
                print('\ngradF:', gradF)
                
            return np.array(gradF)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array, 
                          jac=gradF, 
                          method="BFGS")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)
            
            
class ETC_optimizeFTPR_F_db(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='T-F-PR+F',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
            
    def fit(self, 
            X_train,
            X_sen_train,
            y_train,
            start_thresholds,
            b_sigmoid=100, 
            sum_threshold=0.8,
            c=0.5,
            mu=0.1,
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the db. approximation of the objective F = -TPR + FPR 
        over these thresholds.
        under some db. approximation covariance based side constraints, 
        put into a log barrier penalty.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of training sensitive (feature) inputs 
                            of shape samples x sensitive features
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        b_sigmoid:          positive float that indicates 
                            the b in the sigmoid function
        sum_thresholds:     positive float that indicates
                            when the sum of sensor node outputs indicates a leak
        c:                  positive float that indicates
                            the upper bound of the absolute value
                            of the side constraint(s)
        mu:                 positive float that indicates
                            the hyperparameter of the log barrier method                    
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective and objective's gradient
        def F(thresholds_array):
            _, y_pred = self.predict_approx(X_train, 
                                            keys_list=list(start_thresholds.keys()),
                                            thresholds_array=thresholds_array, 
                                            b_sigmoid=b_sigmoid,
                                            sum_threshold=sum_threshold)
            F = -TPR(y_train,y_pred) + FPR(y_train,y_pred)
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective and fairness information:')
                print('TPR:', TPR(y_train,y_pred))
                print('FPR:', FPR(y_train,y_pred))
                print('F:  ', F)
            
            sensitive_features = list(X_sen_train.columns)
            for sensitive_feature in sensitive_features:
                x_sen_train = X_sen_train.loc[:, sensitive_feature]
                cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                F -= mu * log_on_R(c - cov)
                F -= mu * log_on_R(c + cov)
                
                if print_coeff:
                    print('Group:', sensitive_feature)
                    print('  cov:', cov)
                    print('  - mu * log_on_R(c - cov):', - mu * log_on_R(c - cov))
                    print('  - mu * log_on_R(c + cov):', - mu * log_on_R(c + cov))
                    print('  F:', F)
                    
            if print_coeff:
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
            
            return np.array(F)
            
        def gradF(thresholds_array):
            _, y_pred = self.predict_approx(X_train, 
                                            keys_list=list(start_thresholds.keys()),
                                            thresholds_array=thresholds_array, 
                                            b_sigmoid=b_sigmoid,
                                            sum_threshold=sum_threshold)
            dx_Y_pred = self.dx_predict_approx(X_train,
                                               keys_list=list(start_thresholds.keys()),
                                               thresholds_array=thresholds_array, 
                                               b_sigmoid=b_sigmoid,
                                               sum_threshold=sum_threshold)
            gradF = list()
            sensor_ids = list(X_train.columns)
            for node in sensor_ids:
                dx_y_pred = dx_Y_pred.loc[:,node]
                dx_y_pred = pd.DataFrame(data=dx_y_pred,
                                         index=dx_y_pred.index)
                dx_y_pred = dx_y_pred.rename({node:'y'},
                                              axis='columns')
                partialF = -TPR(y_train,dx_y_pred) + FPR(y_train,dx_y_pred)
                
                sensitive_features = list(X_sen_train.columns)
                for sensitive_feature in sensitive_features:
                    x_sen_train = X_sen_train.loc[:, sensitive_feature]
                    cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                    dx_cov = Cov(x_sen_train, dx_y_pred.loc[:,'y'])
                    partialF -= (mu * -1 * dx_cov) / (c - cov)
                    partialF -= (mu * dx_cov) / (c + cov)
                gradF.append(partialF)
                
            if print_coeff:
                print('\ngradF:', gradF)
                
            return np.array(gradF)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array, 
                          jac=gradF, 
                          method="BFGS")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)
        
class ETC_optimizeFTPR_ndb(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='T-F-PR-ndb',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
            
    def fit(self, 
            X_train, 
            X_sen_train,
            y_train,
            start_thresholds,
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the exact objective F = -TPR + FPR 
        over these thresholds.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of training sensitive (feature) inputs 
                            of shape samples x sensitive features
                            (only required for evaluation if print_coeff=True)
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective 
        def F(thresholds_array):
            _, y_pred = self.predict(X_train, 
                                     keys_list=list(start_thresholds.keys()),
                                     thresholds_array=thresholds_array)
            F = -TPR(y_train,y_pred) + FPR(y_train,y_pred)
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective information:')
                print('TPR:', TPR(y_train,y_pred))
                print('FPR:', FPR(y_train,y_pred))
                print('F:  ', F)
                
                print('\nCurrent fairness information:')
                sensitive_features = list(X_sen_train.columns)
                for sensitive_feature in sensitive_features:
                    x_sen_train = X_sen_train.loc[:, sensitive_feature]
                    cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                    print('Group:', sensitive_feature)
                    print('  cov:', cov)
        
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
                
            return np.array(F)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array,  
                          method="Nelder-Mead")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)
        
class ETC_optimizeFTPR_F_ndb(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='T-F-PR+F-ndb',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
            
    def fit(self, 
            X_train,
            X_sen_train,
            y_train,
            start_thresholds,
            c=0.5,
            mu=0.1,
            barrier='log',
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the exact objective F = -TPR + FPR 
        over these thresholds
        under some exact covariance based side constraints,
        put into a log barrier or max barrier penalty.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of test sensitive (feature) inputs 
                            of shape samples x sensitive features
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        c:                  positive float that indicates
                            the upper bound of the absolute value
                            of the side constraint(s)
        mu:                 positive float that indicates
                            the hyperparameter of the log or max barrier method  
        barrier:            string of either 'log' or 'max' to choose 
                            the barrier method
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective
        def F(thresholds_array):
            _, y_pred = self.predict(X_train, 
                                     keys_list=list(start_thresholds.keys()),
                                     thresholds_array=thresholds_array)
            F = -TPR(y_train,y_pred) + FPR(y_train,y_pred)
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective and fairness information:')
                print('TPR:', TPR(y_train,y_pred))
                print('FPR:', FPR(y_train,y_pred))
                print('F:  ', F)
            
            sensitive_features = list(X_sen_train.columns)
            for sensitive_feature in sensitive_features:
                x_sen_train = X_sen_train.loc[:, sensitive_feature]
                cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                if barrier == 'log':
                    F -= mu * log_on_R(c - cov)
                    F -= mu * log_on_R(c + cov)
                if barrier == 'max':
                    F += mu * max(0, -(c - cov))
                    F += mu * max(0, -(c + cov))

                if print_coeff:
                    print('Group:', sensitive_feature)
                    print('  cov:', cov)
                    if barrier == 'log':
                        print('  - mu * log_on_R(c - cov):', - mu * log_on_R(c - cov))
                        print('  - mu * log_on_R(c + cov):', - mu * log_on_R(c + cov))
                    if barrier == 'max':
                        print('  + mu * max(0, -(c - cov)):', mu * max(0, -(c - cov)))
                        print('  + mu * max(0, -(c + cov)):', mu * max(0, -(c + cov)))
                    print('  F:', F)
                    
            if print_coeff:
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
            
            return np.array(F)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array,  
                          method="Nelder-Mead")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)
        
class ETC_optimizeACC_db(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='ACC',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
            
    def fit(self, 
            X_train, 
            X_sen_train,
            y_train,
            start_thresholds,
            b_sigmoid=100, 
            sum_threshold=0.8,
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the db. approximation of the objective F = -AUC
        over these thresholds.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of training sensitive (feature) inputs 
                            of shape samples x sensitive features
                            (only required for evaluation if print_coeff=True)
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        b_sigmoid:          positive float that indicates 
                            the b in the sigmoid function
        sum_thresholds:     positive float that indicates
                            when the sum of sensor node outputs indicates a leak
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective and objective's gradient
        def F(thresholds_array):
            _, y_pred = self.predict_approx(X_train, 
                                            keys_list=list(start_thresholds.keys()),
                                            thresholds_array=thresholds_array, 
                                            b_sigmoid=b_sigmoid,
                                            sum_threshold=sum_threshold)
            F = -ACC(y_train,y_pred)
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective information:')
                print('ACC:', ACC(y_train,y_pred))
                print('F:  ', F)
                
                print('\nCurrent fairness information:')
                sensitive_features = list(X_sen_train.columns)
                for sensitive_feature in sensitive_features:
                    x_sen_train = X_sen_train.loc[:, sensitive_feature]
                    cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                    print('Group:', sensitive_feature)
                    print('  cov:', cov)
        
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
                
            return np.array(F)
            
        def gradF(thresholds_array):
            dx_Y_pred = self.dx_predict_approx(X_train,
                                               keys_list=list(start_thresholds.keys()),
                                               thresholds_array=thresholds_array, 
                                               b_sigmoid=b_sigmoid,
                                               sum_threshold=sum_threshold)
            gradF = list()
            sensor_ids = list(X_train.columns)
            for node in sensor_ids:
                dx_y_pred = dx_Y_pred.loc[:,node]
                dx_y_pred = pd.DataFrame(data=dx_y_pred,
                                         index=dx_y_pred.index)
                dx_y_pred = dx_y_pred.rename({node:'y'},
                                              axis='columns')
                partialF = -dx_ACC(y_train,dx_y_pred)
                gradF.append(partialF)
                
            if print_coeff:
                print('\ngradF:', gradF)  
                
            return np.array(gradF)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array, 
                          jac=gradF, 
                          method="BFGS")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)
            
            
class ETC_optimizeACC_F_db(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='ACC+F',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
            
    def fit(self, 
            X_train,
            X_sen_train,
            y_train,
            start_thresholds,
            b_sigmoid=100, 
            sum_threshold=0.8,
            c=0.5,
            mu=0.1,
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the db. approximation of the objective F = -AUC
        over these thresholds
        under some db. approximation covariance based side constraints, 
        put into a log barrier penalty.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of training sensitive (feature) inputs 
                            of shape samples x sensitive features
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        b_sigmoid:          positive float that indicates 
                            the b in the sigmoid function
        sum_thresholds:     positive float that indicates
                            when the sum of sensor node outputs indicates a leak
        c:                  positive float that indicates
                            the upper bound of the absolute value
                            of the side constraint(s)
        mu:                 positive float that indicates
                            the hyperparameter of the log barrier method                    
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective and objective's gradient
        def F(thresholds_array):
            _, y_pred = self.predict_approx(X_train, 
                                            keys_list=list(start_thresholds.keys()),
                                            thresholds_array=thresholds_array, 
                                            b_sigmoid=b_sigmoid,
                                            sum_threshold=sum_threshold)
            F = -ACC(y_train,y_pred)
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective and fairness information:')
                print('ACC:', ACC(y_train,y_pred))
                print('F:  ', F)
            
            sensitive_features = list(X_sen_train.columns)
            for sensitive_feature in sensitive_features:
                x_sen_train = X_sen_train.loc[:, sensitive_feature]
                cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                F -= mu * log_on_R(c - cov)
                F -= mu * log_on_R(c + cov)
                
                if print_coeff:
                    print('Group:', sensitive_feature)
                    print('  cov:', cov)
                    print('  - mu * log_on_R(c - cov):', - mu * log_on_R(c - cov))
                    print('  - mu * log_on_R(c + cov):', - mu * log_on_R(c + cov))
                    print('  F:', F)
                    
            if print_coeff:
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
            
            return np.array(F)
            
        def gradF(thresholds_array):
            _, y_pred = self.predict_approx(X_train, 
                                            keys_list=list(start_thresholds.keys()),
                                            thresholds_array=thresholds_array, 
                                            b_sigmoid=b_sigmoid,
                                            sum_threshold=sum_threshold)
            dx_Y_pred = self.dx_predict_approx(X_train,
                                               keys_list=list(start_thresholds.keys()),
                                               thresholds_array=thresholds_array, 
                                               b_sigmoid=b_sigmoid,
                                               sum_threshold=sum_threshold)
            gradF = list()
            sensor_ids = list(X_train.columns)
            for node in sensor_ids:
                dx_y_pred = dx_Y_pred.loc[:,node]
                dx_y_pred = pd.DataFrame(data=dx_y_pred,
                                         index=dx_y_pred.index)
                dx_y_pred = dx_y_pred.rename({node:'y'},
                                              axis='columns')
                partialF = -dx_ACC(y_train,dx_y_pred)
                
                sensitive_features = list(X_sen_train.columns)
                for sensitive_feature in sensitive_features:
                    x_sen_train = X_sen_train.loc[:, sensitive_feature]
                    cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                    dx_cov = Cov(x_sen_train, dx_y_pred.loc[:,'y'])
                    partialF -= (mu * -1 * dx_cov) / (c - cov)
                    partialF -= (mu * dx_cov) / (c + cov)
                gradF.append(partialF)
                
            if print_coeff:    
                print('\ngradF:', gradF) 
                
            return np.array(gradF)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array, 
                          jac=gradF, 
                          method="BFGS")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)
        
class ETC_optimizeACC_ndb(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='ACC-ndb',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
            
    def fit(self, 
            X_train, 
            X_sen_train,
            y_train,
            start_thresholds,
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the exact objective F = -AUC
        over these thresholds.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of training sensitive (feature) inputs 
                            of shape samples x sensitive features
                            (only required for evaluation if print_coeff=True)
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective
        def F(thresholds_array):
            _, y_pred = self.predict(X_train, 
                                     keys_list=list(start_thresholds.keys()),
                                     thresholds_array=thresholds_array)
            F = -ACC(y_train,y_pred)
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective information:')
                print('ACC:', ACC(y_train,y_pred))
                print('F:  ', F)
                
                print('\nCurrent fairness information:')
                sensitive_features = list(X_sen_train.columns)
                for sensitive_feature in sensitive_features:
                    x_sen_train = X_sen_train.loc[:, sensitive_feature]
                    cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                    print('Group:', sensitive_feature)
                    print('  cov:', cov)
        
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
            
            return np.array(F)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array,  
                          method="Nelder-Mead")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)
        
class ETC_optimizeACC_F_ndb(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='ACC+F-ndb',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
            
    def fit(self, 
            X_train,
            X_sen_train,
            y_train,
            start_thresholds,
            c=0.5,
            mu=0.1,
            barrier='log',
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the exact objective F = -AUC
        over these thresholds
        under some exact covariance based side constraints,
        put into a log barrier or max barrier penalty.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of training sensitive (feature) inputs 
                            of shape samples x sensitive features
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        c:                  positive float that indicates
                            the upper bound of the absolute value
                            of the side constraint(s)
        mu:                 positive float that indicates
                            the hyperparameter of the log or max barrier method
        barrier:            string of either 'log' or 'max' to choose 
                            the barrier method
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective
        def F(thresholds_array):
            _, y_pred = self.predict(X_train, 
                                     keys_list=list(start_thresholds.keys()),
                                     thresholds_array=thresholds_array)
            F = -ACC(y_train,y_pred)
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective and fairness information:')
                print('ACC:', ACC(y_train,y_pred))
                print('F:  ', F)
            
            sensitive_features = list(X_sen_train.columns)
            for sensitive_feature in sensitive_features:
                x_sen_train = X_sen_train.loc[:, sensitive_feature]
                cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                if barrier == 'log':
                    F -= mu * log_on_R(c - cov)
                    F -= mu * log_on_R(c + cov)
                if barrier == 'max':
                    F += mu * max(0, -(c - cov))
                    F += mu * max(0, -(c + cov))

                if print_coeff:
                    print('Group:', sensitive_feature)
                    print('  cov:', cov)
                    if barrier == 'log':
                        print('  - mu * log_on_R(c - cov):', - mu * log_on_R(c - cov))
                        print('  - mu * log_on_R(c + cov):', - mu * log_on_R(c + cov))
                    if barrier == 'max':
                        print('  + mu * max(0, -(c - cov)):', mu * max(0, -(c - cov)))
                        print('  + mu * max(0, -(c + cov)):', mu * max(0, -(c + cov)))
                    print('  F:', F)
                    
            if print_coeff:
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
            
            return np.array(F)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array,  
                          method="Nelder-Mead")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)
        
class ETC_optimizeCOV_ndb(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='COV+ACC-ndb',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
        
    def fit(self, 
            X_train,
            X_sen_train,
            y_train,
            start_thresholds,
            mu=0.01,
            lamb=0.04,
            barrier='log',
            acc_best=1,
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the exact objective F = -\sum_k Cov(S_k,\hat{Y})
        over these thresholds
        under some exact accuracy based side constraints,
        put into a log barrier or max barrier penalty.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of test sensitive (feature) inputs 
                            of shape samples x sensitive features
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        mu:                 positive float that indicates
                            the hyperparameter of the log or max barrier method
        lamb:               positive float that indicates
                            the percentage we allow to deviate from acc_best
        barrier:            string of either 'log' or 'max' to choose 
                            the barrier method
        acc_best:           positive float between 0 and 1 that indicates
                            the best accuarcy reach in this optimization problem
                            without considering fairness constraints
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective
        def F(thresholds_array):
            _, y_pred = self.predict(X_train, 
                                     keys_list=list(start_thresholds.keys()),
                                     thresholds_array=thresholds_array)
            F = 0
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective and fairness information:')
            
            sensitive_features = list(X_sen_train.columns)
            for sensitive_feature in sensitive_features:
                x_sen_train = X_sen_train.loc[:, sensitive_feature]
                cov = Cov(x_sen_train, y_pred.loc[:,'y'])
                F += abs(cov)
                
                if print_coeff:
                    print('Group:', sensitive_feature)
                    print('  cov:', cov)
                    print('  F:', F)
                
            acc = ACC(y_train,y_pred)
            if barrier == 'log':
                F -= mu * log_on_R(acc - (1-lamb)*acc_best)
            if barrier == 'max':
                F += mu * max(0, -(acc - (1-lamb)*acc_best))
            
            if print_coeff:
                print('ACC:                                      ', acc)
                print('acc - (1-lamb)*acc_best:                  ', acc - (1-lamb)*acc_best)
                if barrier == 'log':
                    print('- mu * log_on_R(acc - (1-lamb)*acc_best): ', - mu * log_on_R(acc - (1-lamb)*acc_best))
                if barrier == 'max':
                    print('+ mu * max(0, -(acc - (1-lamb)*acc_best)):', mu * max(0, -(acc - (1-lamb)*acc_best)))
                print('F:                                        ', F)
                
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
            
            return np.array(F)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array,  
                          method="Nelder-Mead")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)    
        
class ETC_optimizeEO_ndb(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='EO+ACC-ndb',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
        
    def fit(self, 
            X_train,
            X_sen_train,
            y_train,
            start_thresholds,
            mu=0.1,
            lamb=0.1,
            barrier='log',
            acc_best=1,
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the exact objective F = EO
        over these thresholds
        under some exact accuracy based side constraints,
        put into a log barrier or max barrier penalty.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of test sensitive (feature) inputs 
                            of shape samples x sensitive features
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        mu:                 positive float that indicates
                            the hyperparameter of the log or max barrier method
        lamb:               positive float that indicates
                            the percentage we allow to deviate from acc_best
        barrier:            string of either 'log' or 'max' to choose 
                            the barrier method
        acc_best:           positive float between 0 and 1 that indicates
                            the best accuarcy reach in this optimization problem
                            without considering fairness constraints
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective 
        def F(thresholds_array):
            _, y_pred = self.predict(X_train, 
                                     keys_list=
                                     list(start_thresholds.keys()),
                                     thresholds_array=thresholds_array)
            _,eo,_ = self.score_fairness(X_train,
                                         X_sen_train,
                                         y_train,
                                         keys_list=list(start_thresholds.keys()),
                                         thresholds_array=thresholds_array)
            acc = ACC(y_train,y_pred)
            if barrier == 'log':
                F = eo - mu * log_on_R(acc - (1-lamb)*acc_best)
            if barrier == 'max':
                F = eo + mu * max(0, -(acc - (1-lamb)*acc_best))
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective and fairness information:')
                print('EO:                                       ', eo)
                print('ACC:                                      ', acc)
                print('acc - (1-lamb)*acc_best:                  ', acc - (1-lamb)*acc_best)
                if barrier == 'log':
                    print('- mu * log_on_R(acc - (1-lamb)*acc_best): ', - mu * log_on_R(acc - (1-lamb)*acc_best))
                if barrier == 'max':
                    print('+ mu * max(0, -(acc - (1-lamb)*acc_best)):', mu * max(0, -(acc - (1-lamb)*acc_best)))
                print('F:                                        ', F)
                
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
                
            return np.array(F)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array,  
                          method="Nelder-Mead")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)
        
class ETC_optimizeDI_ndb(EnsembleThresholdClassification):
    
    def __init__(self, 
                 alias='DI+ACC',
                 classifier=ThresholdClassification, 
                 classifier_approx=ThresholdClassificationApproximation):
        super().__init__(classifier, classifier_approx)
        self.alias = alias
        
    def fit(self, 
            X_train,
            X_sen_train,
            y_train,
            start_thresholds,
            mu=0.05,
            lamb=0.04,
            barrier='log',
            acc_best=1,
            print_coeff=False):
        
        """
        Train residual based threshold classification model (leak detector) 
        per sensor node
        based on the (pressure) residual inputs per sensor nodes
        by learning one threshold per sensor node
        by minimizing the exact objective F = -DI
        over these thresholds
        under some exact accuracy based side constraints,
        put into a log barrier or max barrier penalty.
        
        Input:
        X_train:            dataframe of training (pressure) residual inputs 
                            of shape samples x sensors
        X_sen_train         dataframe of test sensitive (feature) inputs 
                            of shape samples x sensitive features
        y_train:            dataframe of training (leak y/n) labels 
                            of shape samples x 1
        start_thresholds:   numpy array of thresholds used for each sensor node
                            of shape nodes x 1
        mu:                 positive float that indicates
                            the hyperparameter of the log or max barrier method
        lamb:               positive float that indicates
                            the percentage we allow to deviate from acc_best
        barrier:            string of either 'log' or 'max' to choose 
                            the barrier method
        acc_best:           positive float between 0 and 1 that indicates
                            the best accuarcy reach in this optimization problem
                            without considering fairness constraints
        print_coeff:        boolean which indicates whether trained coefficients
                            should be printed or not
        """
        
        # --- train one classification model (leak detector) per sensor node
        # --- by minimizing an objective of the ensemble classification model
        
        # -- define objective
        def F(thresholds_array):
            _, y_pred = self.predict(X_train, 
                                     keys_list=
                                     list(start_thresholds.keys()),
                                     thresholds_array=thresholds_array)
            _,_,di = self.score_fairness(X_train,
                                         X_sen_train,
                                         y_train,
                                         keys_list=list(start_thresholds.keys()),
                                         thresholds_array=thresholds_array)
            acc = ACC(y_train,y_pred)
            if barrier == 'log':
                F = - di - mu * log_on_R(acc - (1-lamb)*acc_best)
            if barrier == 'max':
                F = - di + mu * max(0, -(acc - (1-lamb)*acc_best))
            
            if print_coeff:
                print('\n\nCurrent thresholds:')
                for threshold in thresholds_array:
                    print(threshold)
                
                print('\nCurrent objective and fairness information:')
                print('- DI:                                     ', - di)
                print('ACC:                                      ', acc)
                print('acc - (1-lamb)*acc_best:                  ', acc - (1-lamb)*acc_best)
                if barrier == 'log':
                    print('- mu * log_on_R(acc - (1-lamb)*acc_best): ', - mu * log_on_R(acc - (1-lamb)*acc_best))            
                if barrier == 'max':
                    print('+ mu * max(0, -(acc - (1-lamb)*acc_best)):', mu * max(0, -(acc - (1-lamb)*acc_best)))
                print('F:                                        ', F)
                
                print('\nCurrent performance information:')
                tpr,_,fpr,_,_,acc,_ = self.score(X_train,
                                                 y_train,
                                                 keys_list=list(start_thresholds.keys()),
                                                 thresholds_array=thresholds_array)
                _,_,di = self.score_fairness(X_train,
                                             X_sen_train,
                                             y_train,
                                             keys_list=list(start_thresholds.keys()),
                                             thresholds_array=thresholds_array)
                print('TPR, FPR, ACC, DI:', tpr, fpr, acc, di)
            
            return np.array(F)
        
        # -- run minimization of objective
        start_thresholds_array = list(start_thresholds.values())
        start = time.time()
        result = minimize(fun=F, 
                          x0=start_thresholds_array,  
                          method="Nelder-Mead")
        end = time.time()
        print('\n{} (Time: {} sec.)'.format(result.message, end - start))
        thresholds_array = result.x
        self.thresholds = dict(zip(start_thresholds.keys(), thresholds_array))
    
        # -- store trained classification model (leak detector) per sensor node
        self.fit_model(self.thresholds, print_coeff=print_coeff)

In [ ]:
def compute_differences(X_train,
                        y_train,
                        thresholds):
    
    """
    Compute differences between residual and node dependant threshold
    for all residuals with negative label.
    
    Input:
    X_train:            dataframe of training (pressure) residual inputs 
                        of shape samples x sensors
    y_train:            dataframe of training (leak y/n) labels 
                        of shape samples x 1
    thresholds:         dictionary with keys:node, values:(trained) threshold
    
    Output:
    D_train             dataframe of differences between residual and threshold 
                        of shape samples x sensors
    """
    
    D_train = X_train.copy()
    
    # subtract sensor-dependant thresholds of each residual per sensor node
    for sensor in thresholds.keys():
        subtract_threshold = lambda x: x - thresholds[sensor]
        D_train[sensor] = X_train[sensor].apply(subtract_threshold)
    
    # only keep samples with negative label
    filter_noleak = y_train['y'] == 0
    D_train = D_train[filter_noleak]
        
    return D_train

In [ ]:
def find_b_sigmoid(diff,
                   dim,
                   sum_threshold,
                   b_sigmoid):
    
    """
    Function used to estimate the required size of the sigmoid-hyperparameter
    based on characteristics of the training data.
    
    Input:
    diff:            float of a representable difference between residual
                     and threshold of training samples with negative label
    dim:             float of the dimension of sensor values,
                     i.e., number of sensor nodes
    sum_threshold:   float of sum threshold
    b_sigmoid:       float of b in sigmoid function
    """
    
    print('T/d:        ', sum_threshold/dim)
    print('sgd_b(diff):', sigmoid(diff,b_sigmoid))
    print('b large enough:', sigmoid(diff,b_sigmoid) < sum_threshold/dim)

## Evaluation and Visualization of Results

- functionality to filter data according to leak size
- functionality to evaluate a trained multi classification resp. ensemble classification model with respect to performance and fairness
- functionality to visualize the performance of different trained multi classification resp. ensemble classification model with respect to performance and fairness
- functionality to visualize the dependency on the performance score on the fairness scores
- functionality to visualize the dependency on the performance and fairness scores on the training hyperparameters

In [ ]:
def filter_diameter_Hanoi(X_clas,
                          y_clas,
                          X_sen,
                          diameter,
                          df_information,
                          random_state=1):
    
    # find times where leak of diameter is fixed
    filter_start = (df_information['node ID']=='2') & (df_information['diameter']==diameter)
    filter_end = (df_information['node ID']=='32') & (df_information['diameter']==diameter)
    index_start = list(df_information[filter_start].index)[0]
    index_end = list(df_information[filter_end].index)[0]
    time_start_diameter = df_information.loc[index_start, 'setting start ID']
    time_end_diameter = df_information.loc[index_end, 'setting end ID'] 

    split = train_test_split(X_clas.loc[time_start_diameter:time_end_diameter,:], 
                             y_clas.loc[time_start_diameter:time_end_diameter,:], 
                             train_size=0.4,
                             test_size=0.6, 
                             shuffle=True,
                             random_state=random_state, 
                             stratify=y_clas.loc[time_start_diameter:time_end_diameter,:])
    X_clas_train = split[0]
    X_clas_test = split[1]
    y_clas_train = split[2]
    y_clas_test = split[3]
    X_sen_train = X_sen.loc[X_clas_train.index,:]
    X_sen_test = X_sen.loc[X_clas_test.index,:]
    
    return X_clas_train, X_clas_test, y_clas_train, y_clas_test, X_sen_train, X_sen_test

In [ ]:
def filter_diameter_LTown(X_clas,
                          y_clas,
                          X_sen,
                          diameter,
                          df_leaks,
                          random_state=1):
    
    # find times where leak of diameter is fixed
    filter_diameter = df_leaks['dia'] == diameter 

    split = train_test_split(X_clas[filter_diameter], 
                             y_clas[filter_diameter], 
                             train_size=0.4,
                             test_size=0.6, 
                             shuffle=True,
                             random_state=random_state, 
                             stratify=y_clas[filter_diameter])
    X_clas_train = split[0]
    X_clas_test = split[1]
    y_clas_train = split[2]
    y_clas_test = split[3]
    X_sen_train = X_sen.loc[X_clas_train.index,:]
    X_sen_test = X_sen.loc[X_clas_test.index,:]
    
    return X_clas_train, X_clas_test, y_clas_train, y_clas_test, X_sen_train, X_sen_test

In [ ]:
def evaluate(model_clas):
    
    """
    Inputs:   
    model_clas:   trained instance of one of the siblings of the 
                  EnsembleThresholdClassification class
              
    Outputs:
    acc:          float of test accuracy score
    eo:           float of test equal opportunity score
    di:           float of test disparate impact score
    TPRs:         dict with key:sensitive group, 
                  value:test TPR over all samples 
    """
    
    tpr,_,fpr,_,_,acc,_ = model_clas.score(X_clas_train, 
                                           y_clas_train,
                                           print_all_scores=False)
    print('Train TPR:', tpr)
    print('Train FPR:', fpr)
    print('Train ACC:', acc)
    
    tpr,_,fpr,_,_,acc,_ = model_clas.score(X_clas_test, 
                                           y_clas_test,
                                           print_all_scores=False)
    print('Test TPR:', tpr)
    print('Test FPR:', fpr)
    print('Test ACC:', acc)
    
    TPRs,eo,di = model_clas.score_fairness(X_clas_test,
                                           X_sen_test,
                                           y_clas_test,
                                           print_all_scores=False)
    print('Test EO (small):', eo)
    print('Test DI (large):', di)
    
    return acc, eo, di, TPRs

In [ ]:
def graphics_bars(results,
                  rotate=False,
                  title=True,
                  save_figs_d=None):
    
    """
    Inputs:
    results:     dictionary of the structure
                 {model class: {'acc':  float of accuracy,
                              'eo':   float of equal oppotunity,
                              'di':   float of disparate impact,
                              'TPRs': {sensitive feature: float of TPR of sensitive group
                                      }
                              }
                 }
    rotate:      boolean to determine whether to rotate x-axis labels
    title:       boolean to determine qhether to display title
    save_figs_d: string or float to determine whether and under which name
                 the created grahps should be saved
    """

    # --- create dataframes which will be used for plotting
    columns1 = ['method','model','ACC', 'EO','DI','group', 'TPR']
    df1 = pd.DataFrame(data=dict(), columns=columns1)
    columns2 = ['method', 'score type', 'ACC resp. EO']
    df2 = pd.DataFrame(data=dict(), columns=columns2)
    columns3 = ['method', 'score type', 'ACC resp. DI']
    df3 = pd.DataFrame(data=dict(), columns=columns3)
    idx1, idx2, idx3 = 0, 0, 0
    for model_clas in results:
        
        sensitive_features = results[model_clas]['TPRs'].keys()
        for sensitive_feature in sensitive_features:
            df1.loc[idx1,'method'] = model_clas.alias
            df1.loc[idx1,'model'] = model_clas
            df1.loc[idx1,'ACC'] = results[model_clas]['acc']
            df1.loc[idx1,'EO'] = results[model_clas]['eo']
            df1.loc[idx1,'DI'] = results[model_clas]['di']
            df1.loc[idx1,'group'] = sensitive_feature[2:]
            df1.loc[idx1,'TPR'] = results[model_clas]['TPRs'][sensitive_feature]
            idx1 += 1
            
        df2.loc[idx2,'method'] = model_clas.alias
        df2.loc[idx2,'score type'] = 'accuracy'
        df2.loc[idx2,'ACC resp. EO'] = results[model_clas]['acc']
        idx2 +=1
        df2.loc[idx2,'method'] = model_clas.alias
        df2.loc[idx2,'score type'] = 'equal opportunity'
        df2.loc[idx2,'ACC resp. EO'] = results[model_clas]['eo']
        idx2 +=1
        
        df3.loc[idx3,'method'] = model_clas.alias
        df3.loc[idx3,'score type'] = 'accuracy'
        df3.loc[idx3,'ACC resp. DI'] = results[model_clas]['acc']
        idx3 +=1
        df3.loc[idx3,'method'] = model_clas.alias
        df3.loc[idx3,'score type'] = 'disparate impact'
        df3.loc[idx3,'ACC resp. DI'] = results[model_clas]['di']
        idx3 +=1
    
    # --- plot bar plots of TPR, ACC, EO and DI
    # choose seaborn color palette
    color_map = cm.get_cmap(name='Blues', lut=1000)
    sub_color_map = color_map(np.linspace(0.2, 0.9, 3))
    palette = sns.color_palette(palette=sub_color_map)
    
    # true positive rate per method and sensitive group
    if rotate:
        plt.figure(figsize=(5,3.6))
    else:
        plt.figure(figsize=(5,3))
    sns.barplot(x='method', y='TPR', 
                hue='group', 
                data=df1, 
                palette=palette,
                estimator=lambda x: np.mean(x),
                errorbar=lambda x: (x.min(),x.max()))
    plt.legend(loc='lower left', bbox_to_anchor=(0.0, 0.0),
              fontsize='medium')
    if title:
        plt.title('TPR per method and per sensitive group (d={})'.format(save_figs_d),
              fontsize='medium')
    else:
        plt.title(' ')
    if rotate:
        plt.xticks(rotation=30)
    plt.tight_layout()
    if save_figs_d!=None:
        plt.savefig('TPRperMethodAndGroup_d{}.pdf'.format(save_figs_d),
                    format='pdf')
    plt.show()
    
    # accuracy and equal opportunity per method
    if rotate:
        plt.figure(figsize=(5,3.6))
    else:
        plt.figure(figsize=(5,3))
    sns.barplot(x='method', y='ACC resp. EO', 
                hue='score type', 
                data=df2, 
                palette=palette,
                estimator=lambda x: np.mean(x),
                errorbar=lambda x: (x.min(),x.max()))
    plt.ylabel('accuracy and equal opportunity',
              fontsize='medium')
    plt.legend(loc='lower left', bbox_to_anchor=(0.0, 0.0),
              fontsize='medium')
    if title:
        plt.title('Accuracy and equal opportunity per method (d={})'.format(save_figs_d),
                  fontsize='medium')
    else:
        plt.title(' ')
    if rotate:
        plt.xticks(rotation=30)
    plt.tight_layout()
    if save_figs_d!=None:
        plt.savefig('ACCandEOperMethod_d{}.pdf'.format(save_figs_d),
                    format='pdf')
    plt.show()
    
     # accuracy and disparate impact per method
    if rotate:
        fig = plt.figure(figsize=(5,3.6))
    else:
        fig = plt.figure(figsize=(5,3))
    sns.barplot(x='method', y='ACC resp. DI', 
                hue='score type', 
                data=df3, 
                palette=palette,
                estimator=lambda x: np.mean(x),
                errorbar=lambda x: (x.min(),x.max()))
    plt.ylabel('accuracy and disparate impact',
              fontsize='medium')
    plt.legend(loc='lower left', bbox_to_anchor=(0.0, 0.0),
              fontsize='medium')
    if title:
        plt.title('Accuracy and disparate impact per method (d={})'.format(save_figs_d),
                  fontsize='medium')
    else:
        plt.title(' ')
    if rotate:
        plt.xticks(rotation=30)
    plt.tight_layout()
    if save_figs_d!=None:
        plt.savefig('ACCandDIperMethod_d{}.pdf'.format(save_figs_d),
                    format='pdf')
    plt.show()
    
    return df1, df2, df3, fig

In [ ]:
def graphics_scatter(results_fairness,
                     results_nofairness,
                     comparisons,
                     title=True,
                     horizontal=True,
                     save_figs=False):
    
    """
    Inputs:
    results_fairness:   dictionary of the structure
                        {model alias: {diameter: {'ACCs': list of accuracies,
                                                  'EOs':  list of equal opportunities,
                                                  'DIs':  list of disparate impacts,
                                                  'Cs':   list of hyperparameters
                                                  }
                                       }
                         }
    results_nofairness: dictionary of the structure
                        {model alias: {diameter: {'acc': float of accuracy,
                                                  'eo':  float of equal opportunity,
                                                  'di':  float of disparate impact,
                                                  'c':   float of hyperparameter
                                                  }
                                       }
                         }
    comparisons:         dictionary with key:fairness algorithm alias,
                         value:comparison algorithm alias
    title:               boolean to determine whether to display title
    horizontal:          boolean to determine whether to obtain a 3x1 or 1x3 plot
    save_figs:           boolean to determine whether creates grahps should be saved
    """
    
    # choose matplotlib color palette
    nb_colors = len(list(results_fairness[list(results_fairness.keys())[0]].keys()))
    color_map = cm.get_cmap(name='Blues', lut=1000)
    sub_color_map = color_map(np.linspace(0.9, 0.2, nb_colors))
            
    # --- plot scatter plots for coherence of accuracy and equal opportunity  
    if horizontal:
        plt.figure(figsize=(18,5)) 
    else:
        plt.figure(figsize=(5,12))
        
    for i,diameter in enumerate(results_fairness.keys()):
        
        if horizontal:
            plt.subplot(1,3,i+1)
        else:
            plt.subplot(3,1,i+1)
        for j,(color,alias) in enumerate(zip(sub_color_map,results_fairness[diameter].keys())):
            
            # access fairness results
            ACCs = results_fairness[diameter][alias]['ACCs'].copy()
            EOs = results_fairness[diameter][alias]['EOs'].copy()
            # shift the first entry, corresponding to the value (0,0.5)
            # a little bit as else, the point would overlap perfectly
            EOs[0] = EOs[0]+j*0.0025
            
            # access comparison (non-fairness) results
            acc = results_nofairness[diameter][comparisons[alias]]['acc']
            eo = results_nofairness[diameter][comparisons[alias]]['eo']
            
            # accuracy vs. equal opportunity
            plt.scatter([eo],[acc], color=color, marker='x')
            plt.scatter(EOs, ACCs, 
                        color=color,
                        label='d={}, method={}'.format(diameter,
                                                       alias))
        if horizontal:
            size1 = 'large'
            size2 = 'x-large'
            x_pos = 0.51
            y_pos = 0.99
        else:
            size1 = 'medium'
            size2 = 'large'
            x_pos = 0.55
            y_pos = 0.99
            
        plt.xlabel('equal opportunity',
                   fontsize=size1)
        plt.ylabel('accuracy',
                   fontsize=size1)
        plt.legend(fontsize=size1,
                   loc='lower right')
    
    if title:
        plt.suptitle('Coherence of accuracy and equal opportunity',
                     fontsize=size2, x=x_pos, y=y_pos)
    else:
        plt.suptitle(' ',
                     fontsize=size2, x=x_pos, y=y_pos)
    plt.tight_layout()
    if save_figs:
        plt.savefig('ACCandEOCoherence.pdf',
                    format='pdf')
    plt.show()
    
    # --- plot scatter plots for coherence of accuracy and disparate impact
    if horizontal:
        plt.figure(figsize=(18,5)) 
    else:
        plt.figure(figsize=(5,12))
        
    for i,diameter in enumerate(results_fairness.keys()):
        
        if horizontal:
            plt.subplot(1,3,i+1) # remark: three diameters are assumed here
        else:
            plt.subplot(3,1,i+1) # remark: three diameters are assumed here
        for j,(color,alias) in enumerate(zip(sub_color_map,results_fairness[diameter].keys())):
            
            # access fairness results
            ACCs = results_fairness[diameter][alias]['ACCs'].copy()
            DIs = results_fairness[diameter][alias]['DIs'].copy()
            # shift the first entry, corresponding to the value (1,0.5)
            # a little bit as else, the point would overlap perfectly
            DIs[0] = DIs[0]-j*0.0025
            
            # access comparison (non-fairness) results
            acc = results_nofairness[diameter][comparisons[alias]]['acc']
            di = results_nofairness[diameter][comparisons[alias]]['di']
            
            # accuracy vs. disparate impact
            plt.scatter([di],[acc], color=color, marker='x')
            plt.scatter(DIs, ACCs, 
                        color=color,
                        #s=50/(j+1), # changes size in case of overlapping datapoints                                                            
                        label='d={}, method={}'.format(diameter,
                                                       alias))
        if horizontal:
            size1 = 'large'
            size2 = 'x-large'
            x_pos = 0.52
            y_pos = 0.99
        else:
            size1 = 'medium'
            size2 = 'large'
            x_pos = 0.56
            y_pos = 0.99
        
        plt.xlabel('disparate impact',
                   fontsize=size1) 
        plt.ylabel('accuracy', 
                   fontsize=size1)
        plt.legend(fontsize=size1, 
                   loc='lower left')
    
    if title:
        plt.suptitle('Coherence of accuracy and disparate impact',
                     fontsize=size2, x=x_pos, y=y_pos)
    else:
         plt.suptitle(' ',
                 fontsize=size2, x=x_pos, y=y_pos)
    plt.tight_layout()
    if save_figs:
        plt.savefig('ACCandDICoherence.pdf',
                    format='pdf')
    plt.show()

In [ ]:
def graphics_lines(results_fairness,
                   results_nofairness,
                   comparisons,
                   columns='methods',
                   with_eo=False,
                   save_figs=False):
    
    """
    Inputs:
    results_fairness:   dictionary of the structure
                        {model alias: {diameter: {'ACCs': list of accuracies,
                                                  'EOs':  list of equal opportunities,
                                                  'DIs':  list of disparate impacts,
                                                  'Cs':   list of hyperparameters
                                                  }
                                       }
                        }
    results_nofairness: dictionary of the structure
                        {model alias: {diameter: {'acc': float of accuracy,
                                                  'eo':  float of equal opportunity,
                                                  'di':  float of disparate impact,
                                                  'c':   float of hyperparameter
                                                  }
                                       }
                         }
    comparisons:         dictionary with key:fairness algorithm alias,
                         value:comparison algorithm alias
    columns:             string of either 'methods' or 'diameters' to determine if the plot is
                         of size nb_diameters x nb_methods (columns='methods') or
                         of size nb_methods x nb_diameters (columns='diameters')
    with_eo:             boolean to determine whether 
                         the equal opportunity should be plotted
    save_figs:           boolean to determine whether 
                         created grahps should be saved
    """
    
    # --- plot line plots for coherence of 
    # --- accuracy, equal opportunity and disparate impact, resp.,
    # --- and the hyperparameter
    
    # choose matplotlib color palette
    color_map = cm.get_cmap(name='Blues', lut=1000)
    sub_color_map = color_map(np.linspace(0.9, 0.2, 3))
    
    if with_eo:
        plt.figure(figsize=(20,25)) # remark: this could be optimized wrt. nb_diameters,
                                    #         nb_methods and the columns variable.
    else:    
        plt.figure(figsize=(20,15)) # remark: this could be optimized wrt. nb_diameters,
                                    #         nb_methods and the columns variable.
    
    nb_diameters = len(list(results_fairness.keys()))
    nb_methods = len(list(results_fairness[list(results_fairness.keys())[0]].keys()))
    for i, diameter in enumerate(results_fairness.keys()):
    
        for j, alias in enumerate(results_fairness[diameter].keys()):
            
            # access fairness results
            ACCs = results_fairness[diameter][alias]['ACCs']
            EOs = results_fairness[diameter][alias]['EOs']
            DIs = results_fairness[diameter][alias]['DIs']
            if 'Cs' in list(results_fairness[diameter][alias].keys()):
                Cs = results_fairness[diameter][alias]['Cs']
                hyperparameter = 'covariance hyperparameter'
            if 'Lambdas' in list(results_fairness[diameter][alias].keys()):
                Cs = results_fairness[diameter][alias]['Lambdas']
                hyperparameter = 'lambda hyperparameter'
                
            if columns=='methods':
                plt.subplot(nb_diameters,nb_methods,(j+1)+i*nb_methods)
            if columns=='diameters':
                plt.subplot(nb_methods,nb_diameters,(i+1)+j*nb_diameters)
            
            # accuracy vs. hyperparameter
            plt.plot(Cs, ACCs, 
                     color=sub_color_map[0],
                     label='accuracy')
            plt.scatter(Cs, ACCs, 
                        color=sub_color_map[0],
                        marker='x')
            
            # equal opportunity vs. hyperparameter
            if with_eo:
                plt.plot(Cs, EOs, 
                         color=sub_color_map[2],
                         label='equal opportunity')
                plt.scatter(Cs, EOs, 
                             color=sub_color_map[2],
                             marker='x')
            
            # disparate impact vs. hyperparameter
            plt.plot(Cs, DIs, 
                    color=sub_color_map[1],
                     label='disparate impact')
            plt.scatter(Cs, DIs, 
                        color=sub_color_map[1],
                        marker='x')
            
            plt.xticks(rotation=45)
            plt.xlabel(hyperparameter,
                       fontsize='large')
            if with_eo:
                plt.ylabel('accuracy, equal opportunity and disparate impact',
                           fontsize='large')
            else:
                plt.ylabel('accuracy and disparate impact',
                           fontsize='large')
            plt.legend(fontsize='large')
            plt.title('d={}, method={}'.format(diameter,
                                               alias),
                      fontsize='large')
    
    if with_eo:
        plt.suptitle('Accuracy, equal opportunity and disparate impact\n'\
                     'in dependence on the method and its hyperparameter',
                     fontsize='x-large',
                     x=0.52, y=0.99)
        plt.tight_layout()
        if save_figs:
            plt.savefig('ACCandEOandDIvsHyperparameterCoherence.pdf',
                        format='pdf')
    else:
        plt.suptitle('Accuracy and disparate impact\n'\
                     'in dependence on the method and its hyperparameter', 
                     fontsize='x-large',
                     x=0.52, y=0.99)
        plt.tight_layout()
        if save_figs:
            plt.savefig('ACCandDIvsHyperparameterCoherence.pdf',
                        format='pdf')
    plt.tight_layout()
    plt.show()